<a href="https://colab.research.google.com/github/BitLorax/depth-prediction/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import os

In [0]:
!pip install -q git+https://github.com/tensorflow/examples.git
from tensorflow_examples.models.pix2pix import pix2pix

# SSH

In [0]:
!rm -rf /root/.ssh
!mkdir /root/.ssh

In [0]:
!tar xvzf ssh.tar.gz

In [0]:
!cp ssh-colab/* /root/.ssh && rm -rf ssh-colab && rm -rf ssh.tar.gz
!chmod 700 /root/.ssh

In [0]:
!touch /root/.ssh/known_hosts
!ssh-keyscan github.com >> /root/.ssh/known_hosts
!chmod 644 /root/.ssh/known_hosts

# Download Data

In [0]:
!git config --global user.email 'willjhliang@gmail.com'
!git config --global user.name 'BitLorax'

In [0]:
!rm -r .git

In [0]:
!git init
!git remote add -f origin git@github.com:BitLorax/depth-prediction.git

!git config core.sparseCheckout true
!echo 'nyuDepth' >> .git/info/sparse-checkout

In [0]:
!git pull origin master

# Load Data

In [0]:
temp = np.load('nyuDepth/data0.npz')['images']
width = temp.shape[1]
height = temp.shape[2]

images = np.empty((0, width, height, 3))
depths = np.empty((0, width, height))

In [0]:
LOAD_START = 0
LOAD_END = 91

for i in range(LOAD_START, LOAD_END, 1):
  data = np.load('nyuDepth/data' + str(i) + '.npz')
  curImgs = data['images']
  curDeps = data['depths']
  images = np.concatenate((images, curImgs), axis=0)
  depths = np.concatenate((depths, curDeps), axis=0)

In [0]:
m = images.shape[0]

random = np.arange(m)
np.random.shuffle(random)
images = images[random]
depths = depths[random]

In [0]:
TRAINING_SPLIT = .8
idx = (int)(TRAINING_SPLIT * m)
imgTrain = images[:idx]
depTrain = depths[:idx]
imgTest = images[idx:]
depTest = depths[idx:]

In [0]:
print(images.shape)
print(depths.shape)
print()
print(imgTrain.shape)
print(depTrain.shape)
print()
print(imgTest.shape)
print(depTest.shape)

In [0]:
train = tf.data.Dataset.from_tensor_slices((imgTrain, depTrain)).repeat()
train = train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test = tf.data.Dataset.from_tensor_slices((imgTest, depTest))

In [0]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)

# Build Model

In [0]:
vgg16 = tf.keras.applications.VGG16(input_shape=(width, height, 3),
                                    include_top=False,
                                    weights='imagenet')
layerNames = ['block1_conv2',
              'block2_conv2',
              'block3_conv3',
              'block4_conv3',
              'block5_conv3'
]
layers = [vgg16.get_layer(name).output for name in layerNames]
downStack = tf.keras.Model(inputs=vgg16.input, outputs=layers)

In [0]:
upStack = [
           pix2pix.upsample(512, 3),
           pix2pix.upsample(256, 3),
           pix2pix.upsample(128, 3),
           pix2pix.upsample(64, 3),
]

In [0]:
def uNet():
  inputs = tf.keras.layers.Input(shape=(width, height, 3))
  x = inputs

  skips = downStack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  for up, skip in zip(upStack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])
  
  last = tf.keras.layers.Conv2D(1, 3, padding='same')
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [0]:
model = uNet()
model.compile(optimizer='adam',
              loss=tf.keras.losses.Huber())

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True)

# Train Model

In [0]:
def display(dList):
  plt.figure(figsize=(10, 10))
  for i in range(len(dList)):
    plt.subplot(1, len(dList), i + 1)
    plt.imshow(dList[i])
    plt.axis('off')
  plt.show()

In [0]:
sampleImg, sampleDep = images[0], depths[0]

In [0]:
def predict():
  display([sampleImg.astype('uint8'), sampleDep, model.predict(sampleImg[tf.newaxis, ...])[0, :, :, 0]])

In [0]:
predict()

In [0]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    predict()

In [0]:
if not os.path.exists('./checkpoints'):
  os.makedirs('./checkpoints')

In [0]:
checkpointCallback = tf.keras.callbacks.ModelCheckpoint('./checkpoints/ckpt{epoch:02d}-{val_loss:2f}.hdf5',
                                     monitor='val_loss',
                                     mode='min',
                                     verbose=1,
                                     save_freq='epoch')

In [0]:
EPOCHS = 50
STEPS_PER_EPOCH = (imgTrain.shape[0] / BATCH_SIZE)
VAL_STEPS = (imgTest.shape[0] / BATCH_SIZE)
hist = model.fit(train,
                 epochs=EPOCHS,
                 steps_per_epoch = STEPS_PER_EPOCH,
                 validation_steps=VAL_STEPS,
                 validation_data=test,
                 callbacks=[DisplayCallback(), checkpointCallback])

# Push Changes

In [0]:
!git status

In [0]:
!git add checkpoints
!git commit -m "Updated checkpoints"
!git push --set-upstream origin master

In [0]:
!git add checkpoints/ckpt50-0.402291.hdf5

In [0]:
!git push --set-upstream origin master

In [0]:
!mv checkpoints/ckpt50-0.402291.hdf5 'drive/My Drive'